In [1]:
import pandas as pd
from neo4j import GraphDatabase
import requests
from tqdm import tqdm

In [2]:
tqdm.pandas()

In [3]:
driver = GraphDatabase.driver(uri="bolt://localhost:7687", auth=('neo4j', 'neo4j'))

# BISOGNA FARE UNA COSA 4 STAGE:
<ol>
    <li>usare i nomi degli autori presi dall'url dell'immagine e vedere di recuperare quelli non presenti in artgraph;</li>
    <li>ci saranno ancora altri lavori che non vengono presi. Proviamo a prendere il nome dell'autore da <code>artwork_info</code> e di rifare l'operazione
    <li>ci saranno ancora altri lavori che non vengono presi. Se l'autore riscontra delle opere via API, vuol dire che bisogna passare a prendere le info puntuali dell'artwork, quindi prendere il campo <code>url</code> e, infine, provare il matching;
    <li>in ultimo, se l'autore non dà alcun risultato, è molto probabile che ci sia un <b>typo</b> . Va riconosciuto, modificato il tutto e rilanciato
</ol>

Get artworks information

In [4]:
artwork_info = pd.read_csv('WikiArt-Emotions/WikiArt-info.tsv', delimiter='\t')

Get name for trying to do a mapping with ArtGraph

In [5]:
artwork_info['name'] = artwork_info['Image URL'].map(lambda x: '_'.join(x.split('/')[-2:]))

Function that checks if a painting is present in ArtGraph

In [6]:
def is_in_artgraph(param, value, driver):
    base_query = f'match (a:Artwork) where a.{param} = "{value}" return count(distinct a) as num'
    with driver.session(database='neo4j') as session:
        ans = session.run(base_query).data()
    return ans[0]['num']

In [7]:
artwork_info['name_in_artgraph'] = artwork_info.progress_apply(lambda x: is_in_artgraph('name', x['name'], driver), axis=1)

100%|██████████| 4119/4119 [07:47<00:00,  8.81it/s]


Taking artworks that are not present in artgraph

In [8]:
unretrieved = artwork_info[artwork_info.name_in_artgraph == 0]

Taking artist name

In [9]:
unretrieved['artist'] = unretrieved['name'].map(lambda x: x.split('_')[0])
unretrieved['artist_1'] = unretrieved['Artist'].map(lambda x: '-'.join(x.lower().split(' ')))

C:\Users\enzos.LAPTOP-RB20LOIL\AppData\Local\Temp\ipykernel_39380\1062937287.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unretrieved['artist'] = unretrieved['name'].map(lambda x: x.split('_')[0])
C:\Users\enzos.LAPTOP-RB20LOIL\AppData\Local\Temp\ipykernel_39380\1062937287.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unretrieved['artist_1'] = unretrieved['Artist'].map(lambda x: '-'.join(x.lower().split(' ')))


In [10]:
unretrieved.drop(['ID', 'Category', 'Artist', 'Year', 'Painting Info URL', 'Artist Info URL', 'name_in_artgraph','Image URL'],
                 axis = 1, inplace=True)

C:\Users\enzos.LAPTOP-RB20LOIL\AppData\Local\Temp\ipykernel_39380\2354085016.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unretrieved.drop(['ID', 'Category', 'Artist', 'Year', 'Painting Info URL', 'Artist Info URL', 'name_in_artgraph','Image URL'],


In [11]:
# function that gets all the paintings of a specific artist
def get_paintings_by_artist(artist_name):
    base_query = 'https://www.wikiart.org/en/App/Painting/PaintingsByArtist?artistUrl={artist}&json=2'
    return requests.get(base_query.format(artist=artist_name)).json()

# function that gets all painting infos for a specific artwork
def get_painting_info(painting_id: int):
    base_query = 'https://www.wikiart.org/en/App/Painting/ImageJson/{painting}'
    return requests.get(base_query.format(painting=painting_id)).json()

# function that gets all url unmatched artworks with respect to wikiart artwork info
def retrieve_artworks_by_url(artist_name, metadata):
    paintings = get_paintings_by_artist(artist_name)
    paintings = map(lambda x: x['contentId'], paintings)
    paintings = list(map(lambda x: f"{get_painting_info(x)['url']}.jpg", paintings))
    return metadata[metadata['name'].isin(paintings)].name.tolist()

# function that gets all unmatched artworks with respect to basic info got by searching just paintings of spceific artwork
def retrieve_artworks(artist_name, metadata):
    paintings = get_paintings_by_artist(artist_name)
    titles = list(map(lambda x: x['title'], paintings))
    names = list(map(lambda x: x['image'].split('/')[-1][:-10], paintings))
    return metadata[(metadata['Title'].isin(titles)) | (metadata['name'].isin(names))].name.tolist()

Group paintings by artist

In [12]:
unretrieved['name'] = unretrieved['name'].map(lambda x: x.split('_')[1])
grouped = unretrieved.groupby(['artist', 'artist_1'])[['Title', 'name']].apply(lambda x: x.values.tolist()).reset_index(name='metadata')

C:\Users\enzos.LAPTOP-RB20LOIL\AppData\Local\Temp\ipykernel_39380\936181770.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unretrieved['name'] = unretrieved['name'].map(lambda x: x.split('_')[1])


Retrieve artworks following **1**.

In [13]:
retrieved = grouped.progress_apply(lambda x: retrieve_artworks(x.artist, pd.DataFrame(x.metadata, columns=['Title', 'name'])), axis=1)

100%|██████████| 501/501 [02:18<00:00,  3.62it/s]


In [14]:
total = unretrieved.groupby(['artist', 'artist_1'])['name'].apply(list).reset_index(name='names')

placing the first retrieved artwroks

In [15]:
total['ret'] = retrieved

Making comparison and underline unretrieved artworks

In [16]:
total['difference'] = total.apply(lambda x: list(set(x.names) - set(x.ret)), axis=1)
total

,artist,artist_1,names,ret,difference
0,aaron-siskind,aaron-siskind,"[feet-102-1957.jpg, jerome-arizona-1949.jpg]","[feet-102-1957.jpg, jerome-arizona-1949.jpg]",[]
1,abraham-manievich,abraham-manievich,[destruction-of-the-ghetto-kiev-1919.jpg],[destruction-of-the-ghetto-kiev-1919.jpg],[]
2,ad-reinhardt,ad-reinhardt,[study-for-a-painting-1938-1.jpg],[study-for-a-painting-1938-1.jpg],[]
3,adolph-menzel,adolph-menzel,[afternoon-in-the-tuileries-gardens-1867(1).jp...,[afternoon-in-the-tuileries-gardens-1867(1).jp...,[]
4,adriaen-brouwer,adriaen-brouwer,"[youth-making-a-face.jpg, fumatore.jpg, the-bi...","[youth-making-a-face.jpg, fumatore.jpg, the-bi...",[]
...,...,...,...,...,...
496,wolfgang-paalen,wolfgang-paalen,[genius-of-the-species-1938.jpg],[genius-of-the-species-1938.jpg],[]
497,wyndham-lewis,wyndham-lewis,[the-crowd-1915.jpg],[the-crowd-1915.jpg],[]
498,xul-solar,xul-solar,"[reptil-que-sube-1920.jpg, pegaso-de-sol-1922....","[reptil-que-sube-1920.jpg, pegaso-de-sol-1922....",[]
499,yannoulis-chalepas,yannoulis-chalepas,"[mercury-venus-and-pegasus-1933.jpg, satyr-pla...","[mercury-venus-and-pegasus-1933.jpg, satyr-pla...",[]


In [17]:
total.difference.map(len).sum()

53

Attualmente ce ne mancano 53

# STAGE 2: USARE L'ALTRO NOME

In [19]:
import itertools
paintings_still_unretrieved = list(itertools.chain(*total.difference.values.tolist()))
unretrieved = unretrieved[unretrieved.name.isin(paintings_still_unretrieved)]

In [20]:
grouped = unretrieved.groupby(['artist', 'artist_1'])[['Title', 'name']].apply(lambda x: x.values.tolist()).reset_index(name='metadata')
retrieved = grouped.progress_apply(lambda x: retrieve_artworks(x.artist_1, pd.DataFrame(x.metadata, columns=['Title', 'name'])), axis=1)

100%|██████████| 28/28 [00:10<00:00,  2.74it/s]


In [21]:
total = unretrieved.groupby(['artist', 'artist_1'])['name'].apply(list).reset_index(name='names')
total['ret'] = retrieved
total['difference'] = total.apply(lambda x: list(set(x.names) - set(x.ret)), axis=1)

In [22]:
total

,artist,artist_1,names,ret,difference
0,andrea-del-verrocchio,andrea-del-verrocchio,"[madonna-and-child.jpg, lorenzo-de-medici-1480...",[],"[portrait-of-a-woman.jpg, madonna-and-child.jp..."
1,anton-raphael-mengs,anton-raphael-mengs,[carlos-iii-1761.jpg],[],[carlos-iii-1761.jpg]
2,benjamin-west,benjamin-west,[the-treaty-of-penn-with-the-indians-1772.jpg],[],[the-treaty-of-penn-with-the-indians-1772.jpg]
3,boris-gregoriev,boris-grigoriev,[portrait-of-vsevolod-meyerhold-1916.jpg],[portrait-of-vsevolod-meyerhold-1916.jpg],[]
4,caspar-david-friedrich,caspar-david-friedrich,[chalk-cliffs-on-r-gen-1818.jpg],[],[chalk-cliffs-on-r-gen-1818.jpg]
5,francisco-goya,francisco-goya,[saturn-devouring-one-of-his-children-1823.jpg],[],[saturn-devouring-one-of-his-children-1823.jpg]
6,george-bousianis,george-bouzianis,[kefali-andra-1915.jpg],[kefali-andra-1915.jpg],[]
7,j-zsef-rippl-r-nai,jozsef-rippl-ronai,[uncle-piacsek-in-front-of-the-black-sideboard...,[uncle-piacsek-in-front-of-the-black-sideboard...,[]
8,jean-auguste-dominique-ingres,jean-auguste-dominique-ingres,[portrait-of-napoléon-on-the-imperial-throne-1...,[],[portrait-of-napoléon-on-the-imperial-throne-1...
9,john-henry-twachtman,john-henry-twachtman,[landscape.jpg],[landscape.jpg],[]


In [23]:
total[total.difference.map(len) != 0]

,artist,artist_1,names,ret,difference
0,andrea-del-verrocchio,andrea-del-verrocchio,"[madonna-and-child.jpg, lorenzo-de-medici-1480...",[],"[portrait-of-a-woman.jpg, madonna-and-child.jp..."
1,anton-raphael-mengs,anton-raphael-mengs,[carlos-iii-1761.jpg],[],[carlos-iii-1761.jpg]
2,benjamin-west,benjamin-west,[the-treaty-of-penn-with-the-indians-1772.jpg],[],[the-treaty-of-penn-with-the-indians-1772.jpg]
4,caspar-david-friedrich,caspar-david-friedrich,[chalk-cliffs-on-r-gen-1818.jpg],[],[chalk-cliffs-on-r-gen-1818.jpg]
5,francisco-goya,francisco-goya,[saturn-devouring-one-of-his-children-1823.jpg],[],[saturn-devouring-one-of-his-children-1823.jpg]
8,jean-auguste-dominique-ingres,jean-auguste-dominique-ingres,[portrait-of-napoléon-on-the-imperial-throne-1...,[],[portrait-of-napoléon-on-the-imperial-throne-1...
10,john-singleton-copley,john-singleton-copley,[the-major-peirson-s-death-1784.jpg],[],[the-major-peirson-s-death-1784.jpg]
13,li-chevalier,li-chevalier,"[arrachement.jpg, 211.jpg]",[],"[arrachement.jpg, 211.jpg]"
15,max-herman-maxy,m.-h.-maxy,"[nude-with-veil-1922.jpg, nude-1924.jpg, human...",[],"[portrait-of-tristan-tzara-1924.jpg, human-con..."
16,otto-mueller,otto-mueller,[otto-mueller-t-nzerin-mit-schleier-von-einem-...,[],[otto-mueller-bauerngeh-ft-in-frankreich-ca191...


# STAGE **3**
## RECURSIVE QUERY BASED TO URL

In [24]:
paintings_still_unretrieved = list(itertools.chain(*total.difference.values.tolist()))
unretrieved = unretrieved[unretrieved.name.isin(paintings_still_unretrieved)]

In [25]:
grouped = unretrieved.groupby(['artist', 'artist_1'])[['Title', 'name']].apply(lambda x: x.values.tolist()).reset_index(name='metadata')
retrieved = grouped.progress_apply(lambda x: retrieve_artworks_by_url(x.artist, pd.DataFrame(x.metadata, columns=['Title', 'name'])), axis=1)

100%|██████████| 17/17 [10:14<00:00, 36.16s/it]


In [26]:
total = unretrieved.groupby(['artist', 'artist_1'])['name'].apply(list).reset_index(name='names')
total['ret'] = retrieved
total['difference'] = total.apply(lambda x: list(set(x.names) - set(x.ret)), axis=1)

In [27]:
total[total.difference.map(len) != 0]

,artist,artist_1,names,ret,difference
0,andrea-del-verrocchio,andrea-del-verrocchio,"[madonna-and-child.jpg, lorenzo-de-medici-1480...",[],"[portrait-of-a-woman.jpg, madonna-and-child.jp..."
3,caspar-david-friedrich,caspar-david-friedrich,[chalk-cliffs-on-r-gen-1818.jpg],[],[chalk-cliffs-on-r-gen-1818.jpg]
4,francisco-goya,francisco-goya,[saturn-devouring-one-of-his-children-1823.jpg],[],[saturn-devouring-one-of-his-children-1823.jpg]
5,jean-auguste-dominique-ingres,jean-auguste-dominique-ingres,[portrait-of-napoléon-on-the-imperial-throne-1...,[portrait-of-napoléon-bonaparte-the-first-coun...,[portrait-of-napoléon-on-the-imperial-throne-1...
8,li-chevalier,li-chevalier,"[arrachement.jpg, 211.jpg]",[],"[arrachement.jpg, 211.jpg]"
9,max-herman-maxy,m.-h.-maxy,"[nude-with-veil-1922.jpg, nude-1924.jpg, human...",[],"[portrait-of-tristan-tzara-1924.jpg, human-con..."
10,otto-mueller,otto-mueller,[otto-mueller-t-nzerin-mit-schleier-von-einem-...,[],[otto-mueller-bauerngeh-ft-in-frankreich-ca191...
11,p-ricl-s-pantazis,pericles-pantazis-,"[mauvaise-recette.jpg, supreme-court-1880.jpg,...",[],"[ath-the-beach-no2.jpg, supreme-court-1880.jpg..."
12,peter-paul-rubens,peter-paul-rubens,[rape-of-the-daughters-of-leucippus.jpg],[],[rape-of-the-daughters-of-leucippus.jpg]
13,pierre-narcisse-guerin,jean-auguste-dominique-ingres,[the-portrait-of-the-chouan-rochejaquelin.jpg],[],[the-portrait-of-the-chouan-rochejaquelin.jpg]


# STAGE 4
Modifichiamo gli artisti in cui c'è un errore di battitura

In [29]:
total.loc[0, 'artist'] = 'andrea-del.verrochio'

In [37]:
artwork_info[(artwork_info.Artist.str.contains('M. H.'))]

,ID,Category,Artist,Title,Year,Image URL,Painting Info URL,Artist Info URL,name,name_in_artgraph
838,577280c3edc2cb3880f26379,Cubism,M. H. Maxy,Nude with Veil,1922,https://uploads5.wikiart.org/images/max-herman...,https://www.wikiart.org/en/m-h-maxy/nude-with-...,https://www.wikiart.org/en/m-h-maxy,max-herman-maxy_nude-with-veil-1922.jpg,0
953,577280c3edc2cb3880f26359,Cubism,M. H. Maxy,Nude,1924,https://uploads4.wikiart.org/images/max-herman...,https://www.wikiart.org/en/m-h-maxy/nude-1924,https://www.wikiart.org/en/m-h-maxy,max-herman-maxy_nude-1924.jpg,0
2456,577280c3edc2cb3880f26317,Cubism,M. H. Maxy,Human Construction,1926,https://uploads0.wikiart.org/images/max-herman...,https://www.wikiart.org/en/m-h-maxy/human-cons...,https://www.wikiart.org/en/m-h-maxy,max-herman-maxy_human-construction-1926.jpg,0
2961,577280c3edc2cb3880f263a9,Cubism,M. H. Maxy,Portrait of Tristan Tzara,1924,https://uploads7.wikiart.org/images/max-herman...,https://www.wikiart.org/en/m-h-maxy/portrait-o...,https://www.wikiart.org/en/m-h-maxy,max-herman-maxy_portrait-of-tristan-tzara-1924...,0
3302,577280c3edc2cb3880f26329,Cubism,M. H. Maxy,Electric Madonna,1926,https://uploads7.wikiart.org/images/max-herman...,https://www.wikiart.org/en/m-h-maxy/electric-m...,https://www.wikiart.org/en/m-h-maxy,max-herman-maxy_electric-madonna-1926.jpg,0
